### START FROM HERE

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
from pathlib import Path
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import features
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
# === 1. Load road network shapefile ===
road_path = Path("C:/Users/Ardo/Desktop/thesis2/BCN_GrafVial_Trams_ETRS89_SHP.shp")
roads_gdf = gpd.read_file(road_path)

# Project to EPSG:25831 (meters)
if roads_gdf.crs is None or roads_gdf.crs.to_epsg() != 25831:
    roads_gdf = roads_gdf.to_crs(epsg=25831)

# Keep only valid line geometries
#roads_gdf = roads_gdf[roads_gdf.geometry.type.isin(["LineString", "MultiLineString"])].copy()

# === 2. Load building footprints ===
buildings_path = Path("C:/Users/Ardo/Desktop/thesis2/Barcelona.geojson")
buildings_gdf = gpd.read_file(buildings_path)

# Project to EPSG:25831 to match roads
if buildings_gdf.crs is None or buildings_gdf.crs.to_epsg() != 25831:
    buildings_gdf = buildings_gdf.to_crs(epsg=25831)

# === 3. Load trees ===
tree_path = Path("C:/Users/Ardo/Desktop/thesis2/bcn_trees.geojson")
trees_gdf = gpd.read_file(tree_path)

# Ensure trees are projected to EPSG:25831
if trees_gdf.crs is None or trees_gdf.crs.to_epsg() != 25831:
    trees_gdf = trees_gdf.to_crs(epsg=25831)

# ✅ Clean height column
trees_gdf["height"] = pd.to_numeric(trees_gdf["height"], errors="coerce")
trees_gdf["height"] = trees_gdf["height"].fillna(0)

# === 4. Load LandUse ===
landuse_path = Path("C:/Users/Ardo/Desktop/thesis2/BCN_UsosSol_MOD.shp")
landuse_gdf = gpd.read_file(landuse_path)

# Project to EPSG:25831 (meters)
if landuse_gdf.crs is None or landuse_gdf.crs.to_epsg() != 25831:
    landuse_gdf = landuse_gdf.to_crs(epsg=25831)

In [ ]:
# === 4. Sample 10 random streets ===
"""target_districts = {"01","02","10"} #06
filtered_roads = roads_gdf[roads_gdf["Distric_D"].isin(target_districts)].copy()
sampled_roads = filtered_roads.sample(n=200, random_state=42).copy()"""

sampled_roads = roads_gdf.sample(n=500, random_state=42).copy()

#sampled_roads = roads_gdf.sample(n=10, random_state=42).copy()

# === 5. Create 256x256 m square patches around street midpoints ===
half_size = 128  # meters (half of 256)
windows = []
ids = []

for _, row in sampled_roads.iterrows():
    geom = row.geometry
    if geom.length == 0:
        continue
    midpoint = geom.interpolate(0.5, normalized=True)
    x, y = midpoint.x, midpoint.y
    square = box(x - half_size, y - half_size, x + half_size, y + half_size)
    windows.append(square)
    ids.append(row["C_Tram"])

windows_gdf = gpd.GeoDataFrame({'C_Tram': ids, 'geometry': windows}, crs=roads_gdf.crs)

# === (Optional) Visual check ===
ax = windows_gdf.plot(facecolor="none", edgecolor="red", linewidth=1, figsize=(10, 10))
roads_gdf.plot(ax=ax, color="black", linewidth=0.5)
buildings_gdf.plot(ax=ax, facecolor="lightgray", edgecolor="gray", linewidth=0.2)
trees_gdf.plot(ax=ax, color="green", markersize=5, alpha=0.5)
plt.title("Randomly Sampled 128×128 m Urban Patches")
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
for i, window in enumerate(windows_gdf.geometry):
    fig, ax = plt.subplots(figsize=(6, 6))

    # === Clip data to patch window ===
    clipped_roads = roads_gdf[roads_gdf.intersects(window)].copy().clip(window)
    clipped_buildings = buildings_gdf[buildings_gdf.intersects(window)].copy().clip(window)
    clipped_trees = trees_gdf[trees_gdf.intersects(window)].copy()

    # === Compute building height (if Z_MAX_VOL and Z_MIN_VOL exist) ===
    if 'Z_MAX_VOL' in clipped_buildings.columns and 'Z_MIN_VOL' in clipped_buildings.columns:
        clipped_buildings["height"] = clipped_buildings["Z_MAX_VOL"] - clipped_buildings["Z_MIN_VOL"]
    else:
        clipped_buildings["height"] = 0

    # === Plot patch boundary ===
    gpd.GeoSeries(window).boundary.plot(ax=ax, color="red", linestyle="--")

    # === Plot buildings colored by height ===
    if not clipped_buildings.empty:
        clipped_buildings.plot(
            ax=ax,
            column="height",
            cmap="viridis",
            linewidth=0.5,
            edgecolor="gray",
            legend=True,
            legend_kwds={"label": "Building Height (m)", "shrink": 0.5},
            vmin=0,
            vmax=100
        )

    # === Plot roads ===
    if not clipped_roads.empty:
        clipped_roads.plot(ax=ax, color="black", linewidth=1)

    # === Plot tree canopies as 3m radius circles ===
    if not clipped_trees.empty:
        tree_canopies = clipped_trees.geometry.buffer(4.0)
        gpd.GeoSeries(tree_canopies).plot(
            ax=ax,
            facecolor="green",
            edgecolor="darkgreen",
            alpha=0.4,
            label="Tree Canopy"
        )

    # === Finalize plot ===
    ax.set_title(f"Patch {i+1} — Buildings + Roads + Trees", fontsize=11)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()



In [ ]:
def export_geojson_with_top_keys(gdf, out_path, patch_id, features_type):
    geojson_dict = json.loads(gdf.to_json())
    geojson_dict["patch_id"] = patch_id
    geojson_dict["features_type"] = features_type
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(geojson_dict, f, indent=2)

# === Output base path ===
output_dir = Path("C:/Users/Ardo/Desktop/thesis2/patches_combined_256")
output_dir.mkdir(exist_ok=True)

# Save C_Tram IDs to CSV
windows_gdf[["C_Tram"]].to_csv(r"C:\Users\Ardo\Desktop\thesis2\patches_combined_256\sampled_c_tram_ids.csv", index=False)

# === Grid resolution and size ===
grid_size = 256  # pixels
patch_size = 256  # meters
pixel_size = patch_size / grid_size  # 1 meter per pixel

for i, window in enumerate(windows_gdf.geometry):
    patch_id = windows_gdf.iloc[i]["C_Tram"]
    patch_name = f"patch_{i+1:04d}_{patch_id}"
    patch_path = output_dir / patch_name
    patch_path.mkdir(exist_ok=True)

    # === Clip data to patch window ===
    clipped_roads = roads_gdf[roads_gdf.intersects(window)].copy().clip(window)
    clipped_buildings = buildings_gdf[buildings_gdf.intersects(window)].copy().clip(window)
    clipped_trees = trees_gdf[trees_gdf.intersects(window)].copy()
    clipped_landuse = landuse_gdf[landuse_gdf.intersects(window)].copy().clip(window)

    # === Compute building heights, if missing ===
    if "height" not in clipped_buildings.columns:
        if 'Z_MAX_VOL' in clipped_buildings.columns and 'Z_MIN_VOL' in clipped_buildings.columns:
            clipped_buildings["height"] = clipped_buildings["Z_MAX_VOL"] - clipped_buildings["Z_MIN_VOL"]
        else:
            clipped_buildings["height"] = 0

    # === Raster transform ===
    minx, miny, maxx, maxy = window.bounds
    transform = from_origin(minx, maxy, pixel_size, pixel_size)

    # === Initialize rasters ===
    dsm = np.zeros((grid_size, grid_size), dtype=np.float32)
    tree_dsm = np.zeros((grid_size, grid_size), dtype=np.float32)
    landuse_raster = np.zeros((grid_size, grid_size), dtype=np.uint8)

    # === Rasterize buildings to DSM ===
    for _, building in clipped_buildings.iterrows():
        if building.geometry.is_valid:
            height = building.get("height", 0)
            if height > 0:
                mask = features.geometry_mask(
                    [building.geometry],
                    transform=transform,
                    invert=True,
                    out_shape=dsm.shape
                )
                dsm[mask] = np.maximum(dsm[mask], height)

    # === Rasterize trees to tree DSM ===
    for _, tree in clipped_trees.iterrows():
        if tree.geometry.is_valid:
            tree_height = tree.get("height", 5)
            if tree_height <= 0:
                continue
            canopy = tree.geometry.buffer(3.0)
            mask = features.geometry_mask(
                [canopy],
                transform=transform,
                invert=True,
                out_shape=tree_dsm.shape
            )
            tree_dsm[mask] = np.maximum(tree_dsm[mask], tree_height)

    # === Rasterize land use from 'Usos' ===
    def usos_to_code(val):
        if not val:
            return 0
        val = str(val).strip().lower()
        if val == "urban_bloc":
            return 1
        elif val == "public_par":
            return 2
        return 0

    if "Usos" in clipped_landuse.columns and clipped_landuse["Usos"].notna().any():
        unique_usos = clipped_landuse["Usos"].dropna().unique()

        shapes = []
        for _, row in clipped_landuse.iterrows():
            geom = row.geometry
            if geom is not None and geom.is_valid:
                code = usos_to_code(row["Usos"])
                if code > 0:
                    shapes.append((geom, code))

        if shapes:
            landuse_raster = features.rasterize(
                shapes=shapes,
                out_shape=landuse_raster.shape,
                transform=transform,
                fill=0,
                dtype="uint8",
                all_touched=False,
                merge_alg=features.MergeAlg.replace
            )

    # === Create combined land use raster ===
    # Step 1: Create building mask
    building_mask = np.zeros_like(landuse_raster, dtype=np.uint8)
    for _, building in clipped_buildings.iterrows():
        if building.geometry.is_valid:
            mask = features.geometry_mask(
                [building.geometry],
                transform=transform,
                invert=True,
                out_shape=building_mask.shape
            )
            building_mask[mask] = 1

    # Step 2: Start with 1 = default ground / urban block
    combined_landuse_raster = np.full_like(landuse_raster, fill_value=1, dtype=np.uint8)

    # Step 3: Set 5 = park wherever landuse == 2
    combined_landuse_raster[landuse_raster == 2] = 5

    # Step 4: Set 2 = building wherever building mask is 1
    combined_landuse_raster[building_mask == 1] = 2

    # === Save all rasters ===
    def write_tif(path, array, dtype, nodata=None):
        with rasterio.open(
            path, 'w',
            driver='GTiff',
            height=array.shape[0], width=array.shape[1], count=1, dtype=dtype,
            crs='EPSG:25831', transform=transform, compress='LZW', nodata=nodata
        ) as dst:
            dst.write(array, 1)

    write_tif(patch_path / "dsm.tif", dsm, 'float32')
    write_tif(patch_path / "cdsm.tif", tree_dsm * (dsm == 0), 'float32')  # canopy DSM
    write_tif(patch_path / "dem.tif", np.zeros_like(dsm), 'float32')  # optional DEM
    write_tif(patch_path / "landuse.tif", landuse_raster, 'uint8', None)
    write_tif(patch_path / "combined_landuse.tif", combined_landuse_raster, 'uint8', None)

    # === Save clipped land use GeoJSON ===
    export_geojson_with_top_keys(clipped_landuse, patch_path / "landuse.geojson", patch_id, "LandUse")

    # === Export GeoJSONs ===
    export_geojson_with_top_keys(clipped_buildings, patch_path / "buildings.geojson", patch_id, "Building")
    export_geojson_with_top_keys(clipped_roads, patch_path / "roads.geojson", patch_id, "Street")
    export_geojson_with_top_keys(clipped_trees, patch_path / "trees.geojson", patch_id, "Tree")
    window_gdf = gpd.GeoDataFrame({'geometry': [window]}, crs=roads_gdf.crs)
    export_geojson_with_top_keys(window_gdf, patch_path / "window.geojson", patch_id, "Window")

    # === Combined GeoJSON ===
    combined_patch_features = []
    for gdf_clip, label in [
        (clipped_buildings, "Building"),
        (clipped_roads, "Street"),
        (clipped_trees, "Tree")
    ]:
        if not gdf_clip.empty:
            gdf_clip = gdf_clip.copy()
            gdf_clip["features_type"] = label
            gdf_clip["patch_id"] = patch_id
            combined_patch_features.append(gdf_clip)

    window_gdf_combined = gpd.GeoDataFrame(
        {'features_type': ["Window"], 'patch_id': [patch_id], 'geometry': [window]},
        crs=roads_gdf.crs
    )
    combined_patch_features.append(window_gdf_combined)

    combined_patch_gdf = gpd.GeoDataFrame(
        pd.concat(combined_patch_features, ignore_index=True),
        geometry="geometry", crs=roads_gdf.crs
    )
    combined_patch_gdf.to_file(patch_path / "patch_combined.geojson", driver="GeoJSON")

    # Export uncropped buildings
    uncropped_buildings = buildings_gdf[buildings_gdf.intersects(window)].copy()
    export_geojson_with_top_keys(
        uncropped_buildings,
        patch_path / "buildings_uncropped.geojson",
        patch_id,
        "Building"
    )

    # === Visualization ===
    fig, ax = plt.subplots(figsize=(6, 6))
    gpd.GeoSeries(window).boundary.plot(ax=ax, color="red", linestyle="--", linewidth=2)
    if not clipped_buildings.empty:
        clipped_buildings.plot(
            ax=ax,
            column="height",
            cmap="viridis",
            linewidth=0.5,
            edgecolor="gray",
            legend=True,
            legend_kwds={"label": "Building Height (m)", "shrink": 0.5}
        )
    if not clipped_roads.empty:
        clipped_roads.plot(ax=ax, color="black", linewidth=1)
    if not clipped_trees.empty:
        tree_canopies = clipped_trees.geometry.buffer(3.0)
        gpd.GeoSeries(tree_canopies).plot(
            ax=ax,
            facecolor="green",
            edgecolor="darkgreen",
            alpha=0.4,
            label="Tree Canopy"
        )
    ax.set_title(f"Patch {i+1} — patch_id: {patch_id}")
    ax.set_axis_off()
    plt.tight_layout()
    plt.savefig(patch_path / "patch.png", dpi=200, bbox_inches='tight')
    plt.close(fig)

    print(f"✔ Saved patch {i+1:04d} and combined land use raster")
